# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [12]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

# Load environment variables
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [13]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [14]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

API key looks good so far


### Choose topic and expertise level

Set **topic** (e.g. Python, Data Science, LLMs, Software Engineering) and **expertise** (`beginner`, `intermediate`, or `expert`). The tutor will tailor the explanation to that level.

In [19]:
# User preferences: choose topic and expertise level
topic = "Python"  # e.g. "Python", "Data Science", "LLMs", "Software Engineering"
expertise = "beginner"  # "beginner" | "intermediate" | "expert"

# Your question
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [24]:
# Build prompts and messages for both models (tutor adapts to topic and expertise)
system_prompt = f"""You are a helpful technical tutor. The user has chosen:
- Topic: {topic}
- Expertise level: {expertise}

Adapt your explanation to their level:
- Beginner: use simple language, minimal jargon, step-by-step, define terms, avoid assumptions.
- Intermediate: assume some familiarity, use standard terminology, moderate detail and depth.
- Expert: be concise, use technical language freely, focus on nuances, edge cases, and trade-offs.

Answer the user's question with an explanation appropriate to their {expertise} level in {topic}.
Structure your answer using headings and subheadings.
"""
user_prompt = "Please give a detailed explanation to the following question: " + question

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]
print(messages)

[{'role': 'system', 'content': "You are a helpful technical tutor. The user has chosen:\n- Topic: Python\n- Expertise level: beginner\n\nAdapt your explanation to their level:\n- Beginner: use simple language, minimal jargon, step-by-step, define terms, avoid assumptions.\n- Intermediate: assume some familiarity, use standard terminology, moderate detail and depth.\n- Expert: be concise, use technical language freely, focus on nuances, edge cases, and trade-offs.\n\nAnswer the user's question with an explanation appropriate to their beginner level in Python.\nStructure your answer using headings and subheadings.\n"}, {'role': 'user', 'content': 'Please give a detailed explanation to the following question: \nPlease explain what this code does and why:\nyield from {book.get("author") for book in books if book.get("author")}\n'}]


In [25]:
# Get gpt-4o-mini to answer, with streaming
stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    part = chunk.choices[0].delta.content or ""
    response += part
    # Optional: clean markdown code fences for display
    display_text = response.replace("```", "").replace("markdown", "")
    update_display(Markdown(display_text), display_id=display_handle.display_id)

# Understanding the Code

The code you provided is a single line of Python code. It uses the `yield from` statement along with a set comprehension. Let’s break down what this code does and clarify each part step-by-step.

## What is `yield from`?

- **Definition**: The `yield from` statement is used in a generator function. A generator is a special type of function that returns an iterator, which allows you to loop over a sequence of values without creating the whole list in memory.
- **Purpose**: When you use `yield from`, it allows your generator to yield all values from another iterable (like a list or a set), which means you can produce values one at a time.

## Breaking Down the Code

### Components of the Code

1. **Set Comprehension**:
   - The code contains a set comprehension `{ ... }`. This is a way to create a set, which is a collection of unique items.
   - Inside the braces `{}`, we see `book.get("author")` which means we are trying to get the value associated with the key `"author"` from each `book`.
   - The part `for book in books` is a loop that goes through each `book` in the `books` collection.
   - `if book.get("author")` checks whether the `"author"` value exists. If it does, it will include that author in the set.
   
   So, in simpler terms, this part `{book.get("author") for book in books if book.get("author")}` creates a set of authors for all books in the `books` list where the author exists.

2. **The `books` variable**:
   - `books` is likely a list (or some other iterable) that contains dictionary items. Each dictionary represents a book, where `"author"` is one of the keys in those dictionaries.

### How It All Works Together

- The entire line of code `yield from {book.get("author") for book in books if book.get("author")}` does the following:
  1. It creates a set of unique authors from the `books` collection.
  2. It yields each author one at a time as part of a generator function. This means if there are several books, the function can produce a list of authors without storing them all at once in memory.

## Why Use This Code?

- **Efficiency**: Using `yield from` with a set comprehension is efficient because it only generates and uses memory for one author at a time rather than creating a whole list of authors upfront.
- **Removing Duplicates**: A set automatically removes any duplicate authors, so each author will only appear once in the result.

## Summary

To summarize:
- The code extracts unique author names from a collection of books.
- It uses `yield from` to create a generator that produces these names one by one.
- This is useful for managing memory efficiently and ensuring only unique authors are returned.

If you have any further questions or need additional clarifications, feel free to ask!

In [26]:
# Get Llama 3.2 to answer
# (Uses same OpenAI-compatible client with base_url pointing at local Ollama)
response_llama = ollama.chat.completions.create(model=MODEL_LLAMA, messages=messages)
reply = response_llama.choices[0].message.content
display(Markdown(reply))

**Understanding Context and Purpose**
=====================================

To fully understand this code, let's break it down into its individual components.

*   **What is `yield from`?**

    The `yield from` statement is a Python syntax that allows you to iterate over another iterable (like a list or another generator) within your own function. This means the inner iterable can be thought of as an iterator.
*   **What is a dictionary comprehension?**

    A dictionary comprehension is a concise way to create dictionaries in Python. It follows the same format as a list comprehension: `[expression for variable in iterable if condition]`.
*   **The `if book.get("author")` part**

    This means only include books that have an "author" key. If we didn't use this, all books would be included.

**Code Deconstructed**
---------------------

Let's now break down the given line of code:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Here's what this line does step by step:

1.  **The dictionary comprehension**

    ```python
{book.get("author") 
for book in books 
if book.get("author")}
```

    This is using the `yield from` statement within a dictionary comprehension (note that this was not valid Python code previously because dictionaries are iterable but they were being used with `yield from` which doesn't work with them as described).

2.  **Creating an iterator over books**

    In this correct way, it's creating an iterator of the author names in your list of books.

### The `yield from` Part

```
yield from {book.get("author") for book in books if book.get("author")}
```

This uses `yield from`, a statement that makes any object `iterable`, return all items from iterable and pass them to the generator function which returns an iterator. Since we defined `{...}` above which was meant to be used with the original correct syntax of `for` loop, I would give full example:

```python
def iterate_books(books):

    for book in books:
        author = yield from {book.get("author")}
            
        # Yield each name here
            
iterate_books(['{"name": "Bob","author": "J.K. Rowling"}',
                 '{"name": "Alice","author": "P.G. Wodehouse"}'])
```

However the code above is still not perfect as to allow this it should be rewritten with a context managers (generator functions).